# Cleaning Process
## Comprehensive Data Description

**Dataset Overview:**  

| Column | Data Type | Description |
|--------|-----------|-------------|
| `vehicle_id` | int | Unique identifier for each autonomous vehicle |
| `timestamp` | object/datetime | Date and time of the measurement |
| `gps_latitude` | float | Latitude position of the vehicle from GPS sensor |
| `gps_longitude` | float | Longitude position of the vehicle from GPS sensor |
| `lidar_points` | int | Number of points detected by the LiDAR sensor |
| `radar_objects` | int | Number of objects detected by the radar sensor |
| `camera_objects` | int | Number of objects detected by the camera sensor |
| `packet_drop_rate` | float | Fraction of lost communication packets |
| `packet_delivery_ratio` | float | Ratio of successfully delivered packets |
| `latency_ms` | float | Communication latency in milliseconds |
| `throughput_kbps` | float | Network throughput in kilobits per second |
| `collision_detected` | int (0 or 1) | Binary indicator if collision occurred |
| `obstacle_detection_accuracy` | float | Accuracy of obstacle detection (0–1) |
| `decision_accuracy` | float | Accuracy of autonomous decision making (0–1) |

**Summary Statistics:**  

- Sensor features (`lidar_points`, `radar_objects`, `camera_objects`) vary depending on environment.  
- Network features (`latency_ms`, `throughput_kbps`, `packet_drop_rate`) vary due to wireless communication conditions.  
- Accuracy features (`obstacle_detection_accuracy`, `decision_accuracy`) mostly range 0.7–0.9.

_________________________________________
# 1. Data Loading

i. Load dataset

ii. Identify Dataset shape

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore

# Load dataset excluding the modified column
df = pd.read_csv("Dirty_Dataset_with_Log.csv", usecols=lambda col: col not in ["modified_column_name", "modified_row_index"])

print("Dataset loaded successfully.")
display(df.head())

# Dataset shape
print(f"Number of Rows: {df.shape[0]}")
print(f"Number of Columns: {df.shape[1]}\n")


Dataset loaded successfully.


,vehicle_id,timestamp,gps_latitude,gps_longitude,lidar_points,radar_objects,camera_objects,packet_drop_rate,packet_delivery_ratio,latency_ms,throughput_kbps,collision_detected,obstacle_detection_accuracy,decision_accuracy
0,1038,01/01/2025 00:00,37.111566,-121.062897,28474.0,37.0,20.0,0.042,0.771,228.0,803.0,1.0,0.84,0.79
1,1083,01/01/2025 00:00,37.531400,-121.999292,25569.0,48.0,32.0,0.009,0.733,209.0,120.0,0.0,0.78,0.82
2,1034,01/01/2025 00:00,37.342874,-121.807894,11304.0,36.0,5.0,0.014,0.822,89.0,993.0,0.0,0.78,0.89
3,1080,01/01/2025 00:00,37.160521,-121.266002,12801.0,40.0,33.0,0.153,0.716,239.0,348.0,0.0,0.83,0.88
4,1008,01/01/2025 00:00,37.797779,-121.473512,26214.0,27.0,44.0,0.212,0.972,288.0,688.0,0.0,0.96,0.75


Number of Rows: 2303
Number of Columns: 14



_______________
# 2. Redundant Data Summary

i. Find Redundant Data based on the same values in the row

ii. Remove the Redundant Data

In [2]:
# Identify FULL duplicates (entire row is identical)
full_duplicates = df[df.duplicated(keep=False)]

print("\nFULL redundant rows:")
display(full_duplicates)

# Remove only TRUE duplicates (entire row same)
df_missing = df.drop_duplicates(keep="first").reset_index(drop=True)

print("\nAfter removing redundant rows:", df_missing.shape)


FULL redundant rows:


,vehicle_id,timestamp,gps_latitude,gps_longitude,lidar_points,radar_objects,camera_objects,packet_drop_rate,packet_delivery_ratio,latency_ms,throughput_kbps,collision_detected,obstacle_detection_accuracy,decision_accuracy
440,1013,01/01/2025 00:07,37.725168,NaN,20540.0,40.0,NaN,0.149,0.936,168.0,551.0,0.0,0.93,0.83
441,1013,01/01/2025 00:07,37.725168,NaN,20540.0,40.0,NaN,0.149,0.936,168.0,551.0,0.0,0.93,0.83
1286,1014,01/01/2025 00:21,37.325322,NaN,14932.0,9.0,38.0,NaN,0.909,225.0,126.0,0.0,0.77,0.77
1287,1014,01/01/2025 00:21,37.325322,NaN,14932.0,9.0,38.0,NaN,0.909,225.0,126.0,0.0,0.77,0.77
1451,1057,01/01/2025 00:24,NaN,-121.460734,13816.0,43.0,NaN,0.237,0.913,297.0,109.0,0.0,0.90,0.80
1452,1057,01/01/2025 00:24,NaN,-121.460734,13816.0,43.0,NaN,0.237,0.913,297.0,109.0,0.0,0.90,0.80



After removing redundant rows: (2300, 14)


_____________
# Missing Values Summary

In [3]:
missing_rows = df_missing[df_missing.isna().any(axis=1)].copy()

# Identify which columns are missing in each row
missing_rows["missing_columns"] = missing_rows.apply(
    lambda row: [col for col in df_missing.columns if pd.isna(row[col])],
    axis=1
)

# Show only key columns
missing_info = missing_rows[["vehicle_id", "timestamp", "missing_columns"]]

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(missing_info)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

total_missing = df_missing.isna().sum().sum()
print("Total missing values in cleaned dataset:", total_missing)

,vehicle_id,timestamp,missing_columns
26,1016,01/01/2025 00:00,[throughput_kbps]
27,1080,01/01/2025 00:00,[latency_ms]
58,1076,01/01/2025 00:00,[collision_detected]
59,1073,01/01/2025 00:00,[packet_drop_rate]
72,1057,01/01/2025 00:01,[decision_accuracy]
77,1087,01/01/2025 00:01,[gps_longitude]
86,1085,01/01/2025 00:01,[gps_latitude]
95,1097,01/01/2025 00:01,[packet_drop_rate]
175,1060,01/01/2025 00:02,[packet_delivery_ratio]
196,1038,01/01/2025 00:03,[collision_detected]


Total missing values in cleaned dataset: 100


# Fill in Missing Values using KNN imputations

In [ ]:
# from sklearn.impute import KNNImputer
# # Get all numeric columns
# numeric_cols = df_missing.select_dtypes(include=['float64', 'int64']).columns.tolist()

# # Remove vehicle_id from numeric columns
# if "vehicle_id" in numeric_cols:
#     numeric_cols.remove("vehicle_id")

# print("Numeric columns to fill:", numeric_cols)

# # Create a copy of the cleaned dataframe for KNN imputation
# df_knn = df_missing.copy()

# # Create KNN imputer with k=5 (recommended)
# imputer = KNNImputer(n_neighbors=5)

# # Apply only to numeric columns
# df_knn[numeric_cols] = imputer.fit_transform(df_knn[numeric_cols])

# df_knn.to_csv("Cleaned_Dataset.csv", index=False)
# print("KNN-filled dataset saved as Cleaned_Dataset.csv")

from sklearn.impute import KNNImputer

int_cols = [
    "lidar_points",
    "radar_objects",
    "camera_objects",
    "latency_ms",
    "throughput_kbps",
    "collision_detected"
]

float_cols = [
    "gps_latitude",
    "gps_longitude",
    "packet_drop_rate",
    "packet_delivery_ratio",
    "obstacle_detection_accuracy",
    "decision_accuracy"
]

# Combine for KNN processing
numeric_cols = int_cols + float_cols

print("Integer columns:", int_cols)
print("Float columns:", float_cols)

# Create a copy for KNN
df_knn = df_missing.copy()

# --- Run KNN ---
imputer = KNNImputer(n_neighbors=5)

df_knn[numeric_cols] = imputer.fit_transform(df_knn[numeric_cols])

# --- Restore the correct data types ---

# Convert integer columns back to whole numbers
for col in int_cols:
    df_knn[col] = df_knn[col].round().astype(int)

# Float columns remain float (no change needed)

# Save final dataset
df_knn.to_csv("Cleaned_Dataset.csv", index=False)
print("KNN-filled dataset saved as Cleaned_Dataset.csv.")


Integer columns: ['lidar_points', 'radar_objects', 'camera_objects', 'latency_ms', 'throughput_kbps', 'collision_detected']
Float columns: ['gps_latitude', 'gps_longitude', 'packet_drop_rate', 'packet_delivery_ratio', 'obstacle_detection_accuracy', 'decision_accuracy']
KNN-filled dataset saved as Cleaned_Dataset.csv with correct integer and float formatting.


# Write report of the filled value

In [5]:
# 1. Identify missing locations BEFORE imputation
missing_locs = []
for row in df_missing.index:
    for col in numeric_cols:
        if pd.isna(df_missing.loc[row, col]):
            missing_locs.append((row, col))

# 2. Create a results table
results = []

for row, col in missing_locs:
    filled = df_knn.loc[row, col]   # value after KNN imputation
    
    results.append({
        "row_index": row,
        "vehicle_id": df_missing.loc[row, "vehicle_id"],
        "timestamp": df_missing.loc[row, "timestamp"],
        "column_imputed": col,
        "filled_value": filled
    })

# 3. Convert to DataFrame for display
imputation_report = pd.DataFrame(results)

print("KNN Imputation Report (Before vs After):")
display(imputation_report)

# Save report
imputation_report.to_csv("KNN_Imputed_Values_Report.csv", index=False)
print("Saved: KNN_Imputed_Values_Report.csv")


KNN Imputation Report (Before vs After):


,row_index,vehicle_id,timestamp,column_imputed,filled_value
0,26,1016,01/01/2025 00:00,throughput_kbps,471.0000
1,27,1080,01/01/2025 00:00,latency_ms,142.0000
2,58,1076,01/01/2025 00:00,collision_detected,0.0000
3,59,1073,01/01/2025 00:00,packet_drop_rate,0.1654
4,72,1057,01/01/2025 00:01,decision_accuracy,0.8500
...,...,...,...,...,...
95,2164,1040,01/01/2025 00:36,collision_detected,0.0000
96,2190,1064,01/01/2025 00:36,obstacle_detection_accuracy,0.8240
97,2202,1020,01/01/2025 00:36,packet_drop_rate,0.1012
98,2288,1029,01/01/2025 00:38,packet_delivery_ratio,0.8240


Saved: KNN_Imputed_Values_Report.csv


____________
# Information of Dataset

In [6]:
print("Cleaned Dataset Info:")
df_knn.info()

print("\nSummary Statistics:")
display(df_knn.describe())

Cleaned Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2300 entries, 0 to 2299
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   vehicle_id                   2300 non-null   int64  
 1   timestamp                    2300 non-null   object 
 2   gps_latitude                 2300 non-null   float64
 3   gps_longitude                2300 non-null   float64
 4   lidar_points                 2300 non-null   int64  
 5   radar_objects                2300 non-null   int64  
 6   camera_objects               2300 non-null   int64  
 7   packet_drop_rate             2300 non-null   float64
 8   packet_delivery_ratio        2300 non-null   float64
 9   latency_ms                   2300 non-null   int64  
 10  throughput_kbps              2300 non-null   int64  
 11  collision_detected           2300 non-null   int64  
 12  obstacle_detection_accuracy  2300 non-null   float64
 

,vehicle_id,gps_latitude,gps_longitude,lidar_points,radar_objects,camera_objects,packet_drop_rate,packet_delivery_ratio,latency_ms,throughput_kbps,collision_detected,obstacle_detection_accuracy,decision_accuracy
count,2300.000000,2300.000000,2300.000000,2300.000000,2300.000000,2300.000000,2300.000000,2300.000000,2300.000000,2300.000000,2300.000000,2300.000000,2300.000000
mean,1049.513478,37.499000,-121.505809,19983.899130,27.277391,26.789130,0.150195,0.850363,156.076087,549.044348,0.052174,0.846967,0.849623
std,28.920949,0.289869,0.294706,5780.863289,12.779745,12.875871,0.085087,0.086613,82.755323,263.700168,0.222426,0.086724,0.088263
min,1000.000000,37.000115,-121.999854,10002.000000,5.000000,5.000000,0.000000,0.700000,10.000000,100.000000,0.000000,0.700000,0.700000
25%,1024.000000,37.243931,-121.754191,14929.000000,16.000000,16.000000,0.076000,0.776000,87.000000,318.000000,0.000000,0.770000,0.770000
50%,1049.000000,37.499082,-121.512347,19942.000000,27.000000,26.500000,0.153000,0.852000,158.000000,546.500000,0.000000,0.850000,0.850000
75%,1075.000000,37.751229,-121.244705,25039.250000,38.000000,38.000000,0.221000,0.928000,225.000000,782.000000,0.000000,0.920000,0.930000
max,1099.000000,37.999677,-121.001123,29983.000000,49.000000,49.000000,0.300000,1.000000,299.000000,999.000000,1.000000,1.000000,1.000000
